# Semantic Search of HM Dresses

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
import pickle
import torch
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, HTML, Markdown
from typing import Tuple, List
import textwrap
import requests
from io import BytesIO

# Semantic Search for H&M Dresses

We load some embeddings that were made previously

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2', device="cpu")

image_path = "https://calvinw.github.io/semantic-search/images/"
pkl_url = "https://calvinw.github.io/semantic-search/combined_embeddings.pkl"

def load_embeddings() -> pd.DataFrame:
    """Load embeddings from pickle file URL and return as DataFrame."""
    print("Downloading embeddings from URL...")
    response = requests.get(pkl_url)
    if response.status_code == 200:
        data = pickle.loads(response.content)
        return pd.DataFrame(data)
    else:
        raise Exception(f"Failed to download pickle file. Status code: {response.status_code}")

def get_image_url(article_id: str) -> str:
    """Generate the full image URL for a given article ID. Padding 0 needed"""
    return f"{image_path}0{article_id}.jpg"

def format_result(rank: int, article_id: str, text: str, similarity: float) -> str:
    """Format a single search result with markdown styling and image."""
    wrapped_text = textwrap.fill(text, width=80)
    image_url = get_image_url(article_id)
    return f"""
### Match #{rank} (Similarity: {similarity:.2%})
**Article ID:** `{article_id}`

![Product Image]({image_url})

**Description:**
{wrapped_text}
"""

def semantic_search(
    query: str, 
    data: pd.DataFrame, 
    model: SentenceTransformer, 
    top_n: int = 5
) -> Tuple[pd.DataFrame, np.ndarray]:
    """
    Perform semantic search on the dataset.
    
    Args:
        query: Search query string
        data: DataFrame containing embeddings and metadata
        model: SentenceTransformer model
        top_n: Number of top results to return
    
    Returns:
        Tuple of (results DataFrame, similarity scores)
    """
    query_embedding = model.encode([query])
    similarities = cosine_similarity(query_embedding, list(data['combined_embeddings']))[0]
    top_indices = similarities.argsort()[-top_n:][::-1]
    return data.iloc[top_indices], similarities[top_indices]

def display_search_results(query: str, results: pd.DataFrame, similarities: np.ndarray):
    """Display search results in a formatted way using IPython display."""
    display(Markdown(f"## Search Results for: '{query}'"))
    
    for idx, (_, row) in enumerate(results.iterrows()):
        result_text = format_result(
            rank=idx + 1,
            article_id=row['article_id'],
            text=row['combined_text'],
            similarity=similarities[idx]
        )
        display(Markdown(result_text))

# Lets load the embeddings data

In [ ]:
print("Loading embeddings...")
data = load_embeddings()
print(f"Loaded {len(data)} items\n")
print(data.head())

# Now we will run some queries

In [ ]:
query = "Something for a day at the beach."
results, similarities = semantic_search(query, data, model, top_n=4)
display_search_results(query, results, similarities)

# We can run a lot of examples

In [ ]:
queries = [

    "Comfortable for casual wear with pockets.",
    "Elegant long black for an evening event.",
    "A cozy winter with long sleeves and a hood.",
    "A sleeveless summer in green color.",
    "A sporty striped that is light and easy to move in.",
    "A playful floral perfect for a spring picnic.",
    "A chic professional suitable for an important business meeting.",
    "A vibrant evening gown that makes a statement at formal events.",
    "A bohemian maxi ideal for a beach vacation.",
    "A lightweight dress for a casual day out that is breathable and relaxed."

    # Specific Occasions & Events
    "Whimsical piece for a garden tea party with vintage-inspired details and lace trim",
    "Artistic design with asymmetrical elements perfect for a gallery opening",
    "Dramatic attire for conducting an orchestra, flowing yet dignified",
    "Something fun for swing dancing with plenty of twirl factor",
    "Mystical garment fitting for a tarot card reader or fortune teller",
    
    # Specific Aesthetics & Vibes
    "Cyberpunk inspired with metallic accents and modern edge",
    "Cottagecore style that belongs in a fairy tale forest",
    "Channeling 1920s art deco glamour with geometric patterns",
    "Minimalist piece inspired by Scandinavian design principles",
    "Looks like a Monet painting brought to life",
    
    # Unique Scenarios
    "Versatile piece for a travel vlogger that looks great on camera and resists wrinkles",
    "Perfect for stargazing on a warm summer evening",
    "Something that would make Mary Poppins jealous for a magical nanny",
    "Suitable for both teaching kindergarten and attending parent conferences",
    "Elegant yet forgiving for a food critic sampling multiple courses",
    
    # Creative Combinations
    "Combining ballerina grace with punk rock edge",
    "Makes you look like a cool librarian who rides a motorcycle",
    "Merging traditional kimono elements with modern streetwear",
    "Perfect for a botanist who splits time between garden work and formal lectures",
    "Professional by day but transforms for salsa dancing at night",
    
    # Mood-Based
    "Embodies the feeling of watching sunrise over the ocean",
    "Captures the cozy essence of drinking hot chocolate by a fireplace",
    "Feels like a sunny day in a lavender field",
    "Has the energy of a midnight jazz club in New Orleans",
    "Channels the peaceful vibe of a zen garden",
    
    # Character-Inspired
    "For a modern-day Alice exploring contemporary wonderlands",
    "Fitting for a witch who runs a successful tech startup",
    "Perfect for a time-traveling archaeologist",
    "Suitable for a romantic poet who also teaches martial arts",
    "For an intergalactic diplomat attending an Earth conference",
    
    # Weather & Environment
    "Transitions seamlessly from misty morning fog to bright afternoon sun",
    "Feels like a cool breeze on a Mediterranean terrace",
    "Works for both desert heat and air-conditioned museums",
    "Handles spontaneous rain showers while staying elegantly composed",
    "Appropriate from mountain cabin to luxury lodge",
    
    # Activity-Specific
    "For urban foraging that combines utility with style",
    "Perfect for impromptu karaoke performances",
    "Suitable for both painting in the studio and attending gallery openings",
    "Works for morning yoga and afternoon business meetings",
    "Ideal for giving a TED talk while remaining comfortably authentic"
]

# Example usage:
for query in queries:
    results, similarities = semantic_search(query, data, model, top_n=4)
    display_search_results(query, results, similarities)
    display(Markdown("---\n"))  # Add separator between queries